In [1]:
import pandas as pd
import json

from openai import OpenAI
client = OpenAI(
    api_key="anything",
    base_url="http://0.0.0.0:4000"
)

prompt_summary = "Summarize the given Input in 30 to 40 words, and in the same language as the Input. Find all the keywords that are relevant to the summary from the input and use them. : {input}"


def get_summary(input, temp:float=0.7):
    prompt = prompt_summary.format(input=input)
    # request sent to model set on litellm proxy, `litellm --model`
    response = client.chat.completions.create(model="llama3.2:1b", temperature = temp, messages = [
        {
            "role": "user",
            "content": f"{prompt}"
        }
    ])
    return response.choices[0].message.content

In [3]:
with open('output.json' , 'r') as f:
    jj = json.loads(f.read())
djj = pd.DataFrame(jj)

In [4]:
#Generarte summaries of documents in range at temperatures 0.0 and 0.7
dj=djj.loc[ (djj['input'].str.len() > 3300) & (djj['input'].str.len() < 3700)].copy()
dj['temp0'] = dj.input.apply(lambda x : get_summary(x, temp=0.0))
dj['temp7'] = dj.input.apply(lambda x : get_summary(x, temp=0.7))


In [5]:
#use the selfcheckgpt like scoring using a LLM

text_mapping = {'yes': 0.0, 'no': 1.0, 'n/a': 0.5}
prompt_score = "Context: {context}\n\nSentence: {sentence}\n\nIs the sentence a good sumamry of the context above? Answer in yes or no only no punctuations either.\n\nAnswer: "

def get_score(input, output, prompt_score:str, temp:float=0.7):
    prompt = prompt_score.format(sentence=input, context=output)
    #messages=[{ "content": f"{prompt}", "role": "user"}] 
    response = client.chat.completions.create(model="gemma2:27b", temperature = temp, messages = [
        { 
            "role": "user",
            "content": f"{prompt}"
        }
    ])
    return response.choices[0].message.content


In [ ]:
dj['oscore'] =     dj.apply((lambda x : get_score(x['input'], x['output'], prompt_score=prompt_score)), axis=1)
dj['oscore'] = dj.oscore.apply( lambda x : text_mapping[x.lower().strip()])

dj['temp0_score'] = dj.apply((lambda x : get_score(x['input'], x['temp0'], prompt_score=prompt_score)), axis=1)
dj['temp0_score'] = dj.temp0_score.apply( lambda x : text_mapping[x.lower().strip()])

In [6]:
prompt_score = "Context: {context}\n\nSentence: {sentence}\n\nIs the sentence a good sumamry of the context above? Answer in yes or no only no punctuations either.\n\nAnswer: "

def get_score(input, output, prompt_score:str, temp:float=0.7):
    prompt = prompt_score.format(sentence=input, context=output)
    #messages=[{ "content": f"{prompt}", "role": "user"}] 
    response = client.chat.completions.create(model="gemma2:27b", temperature = temp, messages = [
        { 
            "role": "user",
            "content": f"{prompt}"
        }
    ])
    return response.choices[0].message.content

In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
def get_sbert_score(sentences):
    embeddings = model.encode(sentences)
    score =  model.similarity(embeddings, embeddings)
    return pd.Series(score[0].tolist()[1:4])

In [15]:
#input is the original document to summarize
#output is the summary
#temp0  and temp7 are summaries  generated at temperature 0.0 and 0.8
dj[['score0', 'score1', 'score2']] = dj.apply((lambda x : get_sbert_score([x.input, x.output, x.temp0, x.temp7])), axis=1)

